In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import cv2
import os
import sys
import tensorflow as tf
from matplotlib import pyplot as plt

In [3]:
import xml.etree.ElementTree as ET  
from xml.dom import minidom

In [4]:
folder = "/home/popoago/fire_detection/furg-fire-dataset-master/"

In [5]:
files = os.listdir("/home/popoago/fire_detection/furg-fire-dataset-master/")

In [6]:
files

['house2.xml',
 'hand_held_nasa_training2.xml',
 'house4.mp4',
 'hand_held_nasa_training.xml',
 'house4.xml',
 'hand_held_nasa_training.mp4',
 'hand_held_winnebago.xml',
 'house5.xml',
 'house3.mp4',
 'barbecue.xml',
 'house2.mp4',
 'annotation_tool.png',
 'house1.xml',
 'house3.xml',
 'case2_car.xml',
 'barbecue.mp4',
 'house5.mp4',
 'case2_car.mp4',
 'house6.mp4',
 'house6.xml',
 'remove']

In [7]:
if not os.path.exists("/home/popoago/fire_detection/models-master/research/images"):
    os.makedirs("/home/popoago/fire_detection/models-master/research/images")
if not os.path.exists("/home/popoago/fire_detection/models-master/research/xmls"):
    os.makedirs("/home/popoago/fire_detection/models-master/research/xmls")    
if not os.path.exists("/home/popoago/fire_detection/models-master/research/images_test"):
    os.makedirs("/home/popoago/fire_detection/models-master/research/images_test")
if not os.path.exists("/home/popoago/fire_detection/models-master/research/xmls_test"):
    os.makedirs("/home/popoago/fire_detection/models-master/research/xmls_test")    

In [8]:
for file in files:
    if file.endswith(".mp4"):
        xml_file = file[:-3]+"xml"
        tree=ET.parse(folder+xml_file)
        root=tree.getroot()
        cap = cv2.VideoCapture(folder+file)
        frame_no=0
        while(cap.isOpened()):
            ret, frame = cap.read()
            if not ret:
                break
            try:
                check=(root[6][frame_no][1][0].text.split()[0])
                
                image_name=file[:-4]+str(frame_no)+".jpg"
                if frame_no%20==0:
                    cv2.imwrite("/home/popoago/fire_detection/models-master/research/images_test/"+image_name,frame)
                else:    
                    cv2.imwrite("/home/popoago/fire_detection/models-master/research/images/"+image_name,frame)

                annotation=ET.Element('annotation')
                folder_et=ET.SubElement(annotation,'folder')
                filename_et=ET.SubElement(annotation,'filename')
                path_et=ET.SubElement(annotation,'path')
                source_et=ET.SubElement(annotation,'source')
                database_et=ET.SubElement(source_et,"database")
                size_et=ET.SubElement(annotation,'size')
                width=ET.SubElement(size_et,"width")
                height=ET.SubElement(size_et,"height")
                depth=ET.SubElement(size_et,"depth")
                segmented_et=ET.SubElement(annotation,'segmented')
                object_et=ET.SubElement(annotation,'object')
                name_et=ET.SubElement(object_et,'name')
                pose_et=ET.SubElement(object_et,'pose')
                truncated_et=ET.SubElement(object_et,'truncated')
                difficult_et=ET.SubElement(object_et,'difficult')
                occluded_et=ET.SubElement(object_et,'occluded')
                bndbox_et=ET.SubElement(object_et,'bndbox')
                xmin=ET.SubElement(bndbox_et,'xmin')
                xmax=ET.SubElement(bndbox_et,'xmax')
                ymin=ET.SubElement(bndbox_et,'ymin')
                ymax=ET.SubElement(bndbox_et,'ymax')

                folder_et.text='images'
                filename_et.text=image_name
                path_et.text="/home/popoago/fire_detection/xmls/" + image_name
                database_et.text="Unknown"
                width.text=root[3].text
                height.text=root[4].text
                depth.text='3'
                segmented_et.text='0'
                name_et.text='fire'
                pose_et.text='Unspecified'
                truncated_et.text='0'
                difficult_et.text='0'
                xmin.text=root[6][frame_no][1][0].text.split()[0]
                ymin.text=root[6][frame_no][1][0].text.split()[1]
                xmax.text=str(int(root[6][frame_no][1][0].text.split()[0]) + int(root[6][frame_no][1][0].text.split()[2]))
                ymax.text=str(int(root[6][frame_no][1][0].text.split()[1]) + int(root[6][frame_no][1][0].text.split()[3]))

                mydata = ET.tostring(annotation)
                if frame_no%20==0:
                    myfile = open("/home/popoago/fire_detection/models-master/research/xmls_test/"+image_name[:-4]+".xml", "wb")  
                    myfile.write(mydata)
                    myfile.close()
                else:
                    myfile = open("/home/popoago/fire_detection/models-master/research/xmls/"+image_name[:-4]+".xml", "wb")  
                    myfile.write(mydata)
                    myfile.close()
            except:
                pass
            frame_no+=1